In [ ]:
import os
import pickle
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights, vgg16
from PIL import Image

import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

In [ ]:
# CIFAR10Dataset class
class CIFAR10Dataset(Dataset):
    def __init__(self, data_path, batch_files, transform=None, augmentation=None):
        self.data_path = data_path
        self.batch_files = batch_files
        self.augmentation = augmentation  # Augmentation for specific labels
        self.transform = transform  # Default transform for all other labels
        self.batch_data = None
        self.batch_labels = None
        self.batch_index = -1
        self.index_map = []
        self._create_index_map()

    def _create_index_map(self):
        """Create a map of global indices to batch indices."""
        for batch_num, batch_file in enumerate(self.batch_files):
            with open(os.path.join(self.data_path, batch_file), 'rb') as f:
                batch = pickle.load(f, encoding='bytes')
                batch_size = len(batch[b'labels'])
                self.index_map.extend([(batch_num, i) for i in range(batch_size)])

    def _load_batch(self, batch_num):
        """Load a batch given its batch number."""
        batch_file = self.batch_files[batch_num]
        with open(os.path.join(self.data_path, batch_file), 'rb') as f:
            batch = pickle.load(f, encoding='bytes')
            self.batch_data = batch[b'data'].reshape(-1, 3, 32, 32)
            self.batch_labels = batch[b'labels']
        self.batch_index = batch_num

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        batch_num, in_batch_idx = self.index_map[idx]

        # Load the batch
        if batch_num != self.batch_index:
            self._load_batch(batch_num)

        # Fetch image and label from the loaded batch
        image = self.batch_data[in_batch_idx]
        label = self.batch_labels[in_batch_idx]

        # Convert to HWC format (Height, Width, Channels)
        image = image.transpose(1, 2, 0)

        # Convert the image to a PIL image
        image = Image.fromarray(image)

        # If augmentation exists, apply augmentation first and skip the ToTensor in transform
        if self.augmentation: # label in [0, 3, 4, 5]
            # Apply augmentation first
            image = self.augmentation(image)

            # If the augmentation already applied ToTensor, we shouldn't apply it again.
            if isinstance(image, torch.Tensor):
                # Only apply other transforms that are NOT ToTensor()
                for t in self.transform.transforms:
                    if not isinstance(t, transforms.ToTensor):
                        image = t(image)
            else:
                # If not tensor, apply the full transform pipeline
                image = self.transform(image)
        else:
            # If no augmentation or other labels, apply the normal transformation
            image = self.transform(image)

        return image, label


In [ ]:
def train_and_evaluate(trainloader, validloader, model, criterion, optimizer, num_epochs=10, patience=5, data_path=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # To store loss and accuracy for each epoch
    train_losses, valid_losses = [], []
    valid_accuracies = []  # To store validation accuracy
    best_valid_acc = 0.0  # Track best validation accuracy
    best_valid_loss = float('inf')  # Track best validation loss for early stopping
    epochs_no_improve = 0  # Counter for early stopping

    # Construct model save path based on data_path
    if data_path:
        model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_with_aug.pkl"
    else:
        model_save_path = "resnet_model_with_aug.pkl"  # Fallback save path if data_path isn't provided

    # Training loop
    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}")  # Debugging line
        model.train()
        running_loss = 0.0
        total_train_samples = 0

        # Train on training data
        progress_bar = tqdm(trainloader, desc=f'Epoch {epoch+1}/{num_epochs} (Train)')
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            total_train_samples += labels.size(0)

        # Compute epoch training loss
        epoch_train_loss = running_loss / total_train_samples
        train_losses.append(epoch_train_loss)
        print(f'Training: Loss: {epoch_train_loss:.4f}')

        # Validation loop
        model.eval()
        running_valid_loss = 0.0
        correct_predictions = 0
        total_valid_samples = 0

        with torch.no_grad():
            progress_bar = tqdm(validloader, desc=f'Epoch {epoch+1}/{num_epochs} (Valid)')
            for inputs, labels in progress_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_valid_loss += loss.item() * inputs.size(0)

                # Get the predicted labels and calculate accuracy
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_valid_samples += labels.size(0)

        # Compute epoch validation loss
        epoch_valid_loss = running_valid_loss / total_valid_samples
        valid_losses.append(epoch_valid_loss)
        print(f'Validation: Loss: {epoch_valid_loss:.4f}')

        # Compute validation accuracy
        epoch_valid_acc = correct_predictions / total_valid_samples * 100
        valid_accuracies.append(epoch_valid_acc)
        print(f'Validation: Accuracy: {epoch_valid_acc:.2f}%')

        # Check if the current model has the best validation accuracy so far
        if epoch_valid_acc > best_valid_acc:
            best_valid_acc = epoch_valid_acc
            print(f'Saving best model at epoch {epoch+1} with validation accuracy {best_valid_acc:.2f}%')

            # Save the model using pickle at the constructed path
            with open(model_save_path, 'wb') as f:
                pickle.dump(model, f)

        # Check for early stopping based on validation loss
        if epoch_valid_loss < best_valid_loss:
            best_valid_loss = epoch_valid_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    # Return the training and validation losses and validation accuracies
    return train_losses, valid_losses, valid_accuracies


In [ ]:
def run_training(augment=False, epochs=10):

    torch.manual_seed(42)

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Path to the dataset in Google Drive
    data_path = '/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/cifar-10-python/cifar-10-batches-py/'

    # Training batch file names (excluding the test batch)
    train_batches = [f'data_batch_{i}' for i in range(1, 6)]

    # Transformation
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Augmentation
    augmentation = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=2),
        transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.02),  # Color jitter
    ])

    if augment:
    # Instantiate dataset with transformations (for training and validation)
      train_dataset = CIFAR10Dataset(data_path, train_batches, transform=transform, augmentation=augmentation)
    else:
    # Instantiate dataset without transformations (for training and validation)
      train_dataset = CIFAR10Dataset(data_path, train_batches, transform=transform)

    # Split the training dataset into train and validation subsets
    train_size = int(0.9 * len(train_dataset))  # Use 90% for training
    valid_size = len(train_dataset) - train_size  # Use 10% for validation
    train_subset, valid_subset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

    # DataLoader for training and validation
    trainloader = DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=4)
    validloader = DataLoader(valid_subset, batch_size=128, shuffle=False, num_workers=4)

    # Instantiate the model (using ResNet-50)
    model = resnet50(weights=ResNet50_Weights.DEFAULT)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-2)

    # Train and validate the model
    train_losses, valid_losses, valid_accuracies = train_and_evaluate(
        trainloader, validloader, model, criterion, optimizer, num_epochs=epochs, patience=5, data_path=data_path
    )

    return model, train_losses, valid_losses, valid_accuracies

In [ ]:
# lets train the model without augmentation
model, train_losses, valid_losses, valid_accuracies = run_training(augment=False, epochs=10)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Reading the CSV file
df = pd.read_csv("/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/training_results_no_augmentation.csv")

# Creating a figure and axis for the plot
fig, ax1 = plt.subplots()

# Plotting Training and Validation Loss on the left axis
ax1.plot(df["Epoch"], df["Training Loss"], label="Training Loss", color="blue")
ax1.plot(df["Epoch"], df["Validation Loss"], label="Validation Loss", color="orange")
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(loc="upper left")

# Creating a second y-axis for the accuracy
ax2 = ax1.twinx()
ax2.plot(df["Epoch"], df["Validation Accuracy"], label="Validation Accuracy", color="green")
ax2.set_ylabel('Validation Accuracy')
ax2.legend(loc="upper right")

# Displaying the plot
plt.title('Training/Validation Loss and Validation Accuracy')
plt.show()

Now check the result first on the validation set.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from torch.utils.data import DataLoader

# Function to test model
def test_model_with_confusion_matrix(model, testloader):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []
    misclassified_images = []
    misclassified_labels = []
    misclassified_preds = []

    with torch.no_grad():
        progress_bar = tqdm(enumerate(testloader), total=len(testloader), desc='Testing')
        for i, (inputs, labels) in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # Append predictions and labels for confusion matrix and metrics
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Identify misclassifications
            for j in range(labels.size(0)):
                if predicted[j] != labels[j]:
                    misclassified_images.append(inputs[j].cpu())
                    misclassified_labels.append(labels[j].cpu().numpy())
                    misclassified_preds.append(predicted[j].cpu().numpy())

    # Generate the confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Print the computed metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot confusion matrix using seaborn with class names
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

    return misclassified_images, misclassified_labels, misclassified_preds

In [ ]:
# Define class names based on the extracted labels
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

# load the valid_subset
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset in Google Drive

# Test the model and display confusion matrix and metrics
testloader = DataLoader(valid_subset, batch_size=128, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_no_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)
misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, testloader)

For the classes 0, 3, 4, 5 the model performed poorly. These classes will be augmented.

Lets print out some misclassified images out of the validation set.

In [ ]:
def display_misclassified_images(images, true_labels, predicted_labels, num_images=9):
    plt.figure(figsize=(5, 5))
    for i in range(min(num_images, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
        plt.title(f"True: {class_names[true_labels[i]]}, Pred: {class_names[predicted_labels[i]]}", fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display misclassified images
display_misclassified_images(misclassified_images, misclassified_labels, misclassified_preds)

Now evaluate the model on the test set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset in Google Drive
data_path = '/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/cifar-10-python/cifar-10-batches-py/'
test_batches = ['test_batch']

# Default transform for other labels (ensure resizing is included)
transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# Define class names based on the extracted labels
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

# Instantiate the test dataset
test_dataset = CIFAR10Dataset(data_path, test_batches, transform=transform)

# Test the model and display confusion matrix and metrics
testloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_no_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)

misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, testloader)

Now train the model again with augmentation on poorly classified labels!


In [ ]:
# lets train the model with augmentation
model, train_losses, valid_losses, valid_accuracies = run_training(augment=True, epochs=10)

In [ ]:
import matplotlib.pyplot as plt

# Reading the CSV file
df_new = pd.read_csv("/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/training_results_augmentation.csv")

# Creating a figure and axis for the plot
fig, ax1 = plt.subplots()

# Plotting Training and Validation Loss on the left axis
ax1.plot(df_new["Epoch"], df_new["Training Loss"], label="Training Loss", color="blue")
ax1.plot(df_new["Epoch"], df_new["Validation Loss"], label="Validation Loss", color="orange")
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(loc="upper left")

# Creating a second y-axis for the accuracy
ax2 = ax1.twinx()
ax2.plot(df_new["Epoch"], df_new["Validation Accuracy"], label="Validation Accuracy", color="green")
ax2.set_ylabel('Validation Accuracy')
ax2.legend(loc="upper right")

# Displaying the plot
plt.title('Training/Validation Loss and Validation Accuracy')
plt.show()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_with_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)

misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, validloader)

In [ ]:
def display_misclassified_images(images, true_labels, predicted_labels, num_images=9):
    plt.figure(figsize=(5, 5))
    for i in range(min(num_images, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
        plt.title(f"True: {class_names[true_labels[i]]}, Pred: {class_names[predicted_labels[i]]}", fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display misclassified images
display_misclassified_images(misclassified_images, misclassified_labels, misclassified_preds)

Now evaluate the model on the test set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset in Google Drive
data_path = '/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/cifar-10-python/cifar-10-batches-py/'
test_batches = ['test_batch']

# Default transform for other labels (ensure resizing is included)
transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# Define class names based on the extracted labels
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

# Instantiate the test dataset
test_dataset = CIFAR10Dataset(data_path, test_batches, transform=transform)

# Test the model and display confusion matrix and metrics
testloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_with_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)

misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, testloader)

In [ ]:
import os
os._exit(00)

Now train the model with the augmention on all classes

In [ ]:
# lets train the model with augmentation on all classes
model, train_losses, valid_losses, valid_accuracies = run_training(augment=True, epochs=10)

In [ ]:
import matplotlib.pyplot as plt


# Reading the CSV file
df_newer = pd.read_csv("/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/training_results_full_augmentation.csv")

# Creating a figure and axis for the plot
fig, ax1 = plt.subplots()

# Plotting Training and Validation Loss on the left axis
ax1.plot(df_newer["Epoch"], df_newer["Training Loss"], label="Training Loss", color="blue")
ax1.plot(df_newer["Epoch"], df_newer["Validation Loss"], label="Validation Loss", color="orange")
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(loc="upper left")

# Creating a second y-axis for the accuracy
ax2 = ax1.twinx()
ax2.plot(df_newer["Epoch"], df_newer["Validation Accuracy"], label="Validation Accuracy", color="green")
ax2.set_ylabel('Validation Accuracy')
ax2.legend(loc="upper right")

# Displaying the plot
plt.title('Training/Validation Loss and Validation Accuracy')
plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_with_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)

misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, validloader)

In [ ]:
def display_misclassified_images(images, true_labels, predicted_labels, num_images=9):
    plt.figure(figsize=(5, 5))
    for i in range(min(num_images, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
        plt.title(f"True: {class_names[true_labels[i]]}, Pred: {class_names[predicted_labels[i]]}", fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display misclassified images
display_misclassified_images(misclassified_images, misclassified_labels, misclassified_preds)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset in Google Drive
data_path = '/content/drive/MyDrive/Artificial_Intelligence_Course_NTUT/cifar-10-python/cifar-10-batches-py/'
test_batches = ['test_batch']

# Default transform for other labels (ensure resizing is included)
transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# Define class names based on the extracted labels
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

# Instantiate the test dataset
test_dataset = CIFAR10Dataset(data_path, test_batches, transform=transform)

# Test the model and display confusion matrix and metrics
testloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load our trained model
model_save_path = '/'.join(data_path.split('/')[:-3]) + "/resnet_model_with_aug.pkl"
with open(model_save_path, 'rb') as f:
    model = pickle.load(f)

model.to(device)

misclassified_images, misclassified_labels, misclassified_preds = test_model_with_confusion_matrix(model, testloader)